<a href="https://colab.research.google.com/github/HackElite-FYP/Legal-Research-Platform-Core/blob/feature%2Fretrieval/retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# install dependencies
!pip install torch transformers pandas tqdm
!pip install pinecone sentence-transformers torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 25.6 MB/s eta 0:00:00


In [6]:
# init variables
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

PROCESSING_FILE_PATH = '/content/drive/MyDrive/FYP/json/cases_2024.json'
PROCESSING_CASE_INDEX = 0
MODELS = ["naver/splade-cocondenser-ensembledistil"]
INDEX_NAME = "hybrid-search"
PINECORN_API_KEY = "pcsk_2NF2b2_29N4efEx4DyAKxrb9PeWsjNKpkcDubQYRr6y4weFUW5X8wq8SCggAFCGp9boz21"
PINECORN_ENV = "us-east-1"

In [9]:
# load to dataframes
import json
import pandas as pd

with open(PROCESSING_FILE_PATH, 'r', encoding='utf-8') as f:
    cases_data = json.load(f)

cases_df = pd.DataFrame(cases_data)
print(cases_df.head())

                                     id filename primaryLang  \
0  d66a6895-c339-4bd0-9992-790b7b5f4a17      cpa        0132   
1  4aaafdf5-8ac9-4086-b62e-485d250b02bb    court          of   
2  f81236b6-7c88-4337-9701-772651a56abe       ca        writ   
3  0fe6fe07-fd7d-4b4c-a5d3-3644e9c56b56      wrt        0201   
4  b655451f-cad0-4cc4-b5ce-6bc81dbbee30     writ         123   

                                                text  wordCount  
0  Page 1 of 11 \n In the cozy appeal of the demo...       2854  
1  CA/HCC 184/2017  \n \n1 | P a g e  \n  IN THE ...       4330  
2  Page 1 of 11 \n IN THE COURT OF APPEAL OF THE ...       3300  
3  Page 1 of 15 \n IN THE COURT OF APPEAL OF THE ...       4121  
4  1 \n IN THE COURT OF APPEAL OF THE DEMOCRATIC ...       3898  


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm
import json

# Load SPLADE model
model_name = MODELS[0]
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.eval()

# SPLADE sparse vector generator
def splade_sparse_vector(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs).logits.squeeze(0)  # [seq_len, vocab_size]
    scores = torch.log(1 + torch.relu(outputs))
    max_scores, _ = torch.max(scores, dim=0)  # Max-pooling across sequence
    non_zero_indices = torch.nonzero(max_scores).squeeze(1).tolist()
    non_zero_values = max_scores[non_zero_indices].tolist()
    tokens = tokenizer.convert_ids_to_tokens(non_zero_indices)
    return {
        "indices": tokens,
        "values": non_zero_values
    }

# Process all texts
sparse_vectors = []
for i in tqdm(range(len(cases_df)-527)):
    sparse = splade_sparse_vector(cases_df.loc[i, 'text'])
    sparse_vectors.append({
        "id": cases_df.loc[i, 'id'],
        "sparse_values": sparse,
        # "metadata": {"text": cases_df.loc[i, 'text']}
    })

# Print one example
print(json.dumps(sparse_vectors[0], indent=2))

tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
 50%|█████     | 1/2 [00:01<00:01,  1.49s/it]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

100%|██████████| 2/2 [00:04<00:00,  2.11s/it]

{
  "id": "d66a6895-c339-4bd0-9992-790b7b5f4a17",
  "sparse_values": {
    "indices": [
      ".",
      "3",
      "5",
      "b",
      "j",
      "v",
      "of",
      "he",
      "as",
      "at",
      "she",
      "or",
      "who",
      "##a",
      "no",
      "said",
      "before",
      "made",
      "now",
      "against",
      "##er",
      "re",
      "high",
      "here",
      "number",
      "##u",
      "general",
      "##on",
      "##k",
      "october",
      "##c",
      "party",
      "2013",
      "got",
      "2014",
      "band",
      "20",
      "king",
      "11",
      "president",
      "order",
      "law",
      "##man",
      "art",
      "country",
      "court",
      "support",
      "##3",
      "##ra",
      "department",
      "##4",
      "case",
      "political",
      "police",
      "san",
      "india",
      "considered",
      "minister",
      "##la",
      "festival",
      "decided",
      "##ma",
      "##el",
      "##ka",
      

In [14]:
#init pinecorn
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(
    api_key=PINECORN_API_KEY,     # Replace with your Pinecone API key
)

# Connect to index
dense_index = pc.Index(f"{INDEX_NAME}-dense")
sparse_index = pc.Index(f"{INDEX_NAME}-sparse")

In [48]:
from sentence_transformers import SentenceTransformer

# Create hybrid records
dense_upserts = []
sparse_upserts = []

for index, df in tqdm(cases_df.iloc[527:].iterrows()):
    doc_id = df.id
    dense_model = SentenceTransformer("intfloat/e5-base-v2")
    dense_vec = dense_model.encode(df.text).tolist()
    sparse_vec = splade_sparse_vector(df.text)

     # Convert 'indices' to a list of integers
    sparse_vec['indices'] = tokenizer.convert_tokens_to_ids(sparse_vec['indices'])

    dense_record = {
        "id": doc_id,
        "values": dense_vec,
        "metadata": {
            "text": df.text
        }
    }
    dense_upserts.append(dense_record)

    sparse_record = {
        "id": doc_id,
        "sparse_values": sparse_vec,
        "metadata": {
            "text": df.text
        }
    }
    sparse_upserts.append(sparse_record)

# Upload to Pinecone
dense_index.upsert(vectors=dense_upserts)
sparse_index.upsert(vectors=sparse_upserts)

2it [00:08,  4.35s/it]


{'upserted_count': 2}

In [51]:
#search
query_text = "In the cozy appeal of the"
query_dense = dense_model.encode(query_text).tolist()
query_sparse = splade_sparse_vector(query_text)

dense_query_response = dense_index.query(
    vector=query_dense,
    top_k=5,
    include_metadata=True
)

# Convert 'indices' to a list of integers before querying
query_sparse['indices'] = tokenizer.convert_tokens_to_ids(query_sparse['indices'])

sparse_query_response = sparse_index.query(
    sparse_vector=query_sparse,
    top_k=5,
    include_metadata=True
)

# Print only the 'id' and 'score' for dense query results if metadata is not available
for match in dense_query_response['matches']:
    print(f"Score: {match['score']:.3f} | ID: {match['id']}")
    # Attempt to access metadata and text fields, handling potential KeyError
    try:
        text = match['metadata']['text']
        # You can customize how you want to display the relevant text here
        # For example, print the first 100 characters:
        print(f"Relevant Text: {text[100:200]}...")
    except KeyError:
        print("Metadata or text field not available for this match.")

# Print only the 'id' and 'score' for sparse query results if metadata is not available
for match in sparse_query_response['matches']:
    print(f"Score: {match['score']:.3f} | ID: {match['id']}")
    # Attempt to access metadata and text fields, handling potential KeyError
    try:
        text = match['metadata']['text']
        # You can customize how you want to display the relevant text here
        # For example, print the first 100 characters:
        print(f"Relevant Text: {text[100:200]}...")
    except KeyError:
        print("Metadata or text field not available for this match.")

Score: 0.675 | ID: 313e5fae-828a-40d0-ac21-489a6e6d05d4
Relevant Text: A  
In the matter of an 
Application in terms of 
Article 140 of the 
Constitution for mandates in 
...
Score: 0.671 | ID: 803f669e-14b7-4154-9edf-ab291a47d21e
Relevant Text:  
 
 
 
 
 
 
 
 
 
 
 
  In the matter of an Application for Writs in the 
nature of Writ of Certio...
Score: 7.160 | ID: 803f669e-14b7-4154-9edf-ab291a47d21e
Relevant Text:  
 
 
 
 
 
 
 
 
 
 
 
  In the matter of an Application for Writs in the 
nature of Writ of Certio...
Score: 5.659 | ID: 313e5fae-828a-40d0-ac21-489a6e6d05d4
Relevant Text: A  
In the matter of an 
Application in terms of 
Article 140 of the 
Constitution for mandates in 
...
